In [1]:
config = {
    "user": "root",
    "password": "swim",
    "database": "SwimmerApp",
    "host": "localhost",
}

In [3]:
import hfpy_utils  # created by Paul Barry
import swim_utils  # created by Paul Barry
import my_utils

import os
import DBcm
from datetime import datetime, timedelta

FOLDER = r"swimdata2/"


In [6]:
with DBcm.UseDatabase(config) as db:
    for filename in os.listdir(FOLDER):
        name, age, distance, stroke = filename.removesuffix(".txt").split("-")

        SQL = """INSERT INTO swimmers (swimmer_name, swimmer_age)
                SELECT %s, %s
                WHERE NOT EXISTS (
                    SELECT 1 
                    FROM swimmers 
                    WHERE swimmer_name = %s AND swimmer_age = %s
                )"""
        db.execute(SQL, (name, age, name, age))
        results = db.fetchall()
        SQL = """ INSERT INTO events (event_distance, event_stroke)
                SELECT %s, %s
                WHERE NOT EXISTS (
                    SELECT 1 
                    FROM events 
                    WHERE event_distance = %s AND event_stroke = %s
                )"""
        db.execute(SQL, (distance, stroke, distance, stroke))
        results = db.fetchall()

In [21]:
with DBcm.UseDatabase(config) as db:
    for filename in os.listdir(FOLDER):
        name, age, distance, stroke = filename.removesuffix(".txt").split("-")

        SQL = """SELECT swimmer_ID FROM swimmers WHERE swimmer_name = %s AND swimmer_age = %s"""
        db.execute(SQL, (name, age))
        swimmerID = db.fetchone()


        SQL = """SELECT event_ID FROM events WHERE event_distance = %s AND event_stroke = %s"""
        db.execute(SQL, (distance, stroke))
        event_ID = db.fetchone()

        now = datetime.now()
        
        with open(os.path.join(FOLDER, filename)) as fh:
            data = fh.read()
        times = data.strip().split(",")

        for t in times:
                SQL = """INSERT INTO times (swimmer_ID, event_ID, recorded_time, ts) VALUES (%s, %s, %s, %s)"""
                db.execute(SQL, (swimmerID[0], event_ID[0], t, now))



